In [ ]:
import pandas as pd
import os 
import sqlite3

path_db = "banco_de_dados.db"

conn = sqlite3.connect(path_db)
df_cliente = pd.read_sql_query("SELECT * FROM clientes", conn) 
df_conta = pd.read_sql_query("SELECT * FROM contas", conn)

#df_completo = pd.merge(df_conta, df_cliente, left_on="cpf_cliente", right_on="cpf")

#df_completo = df_completo.drop(columns=["cpf_cliente"])






In [ ]:
# Corrigindo as 3 primeiras linhas do Cpf para ficar no formato padrão (placeholder = 000.000.000-00)
    # Aqui defini uma lista para guardar as infos de como quero que fique os três primeiros cpf
novo_cpf = ["615.019.033-14" , "123.456.789-10","013.179.063.30"]
cpf_alvo = df_cliente['cpf'].head(3).tolist()
# Loop para percorrer e atualizar
for antigo, novo in zip(cpf_alvo, novo_cpf) : 
    df_cliente.loc[df_cliente['cpf']==antigo, 'cpf'] = novo

    df_conta.loc[df_conta['cpf_cliente'] == antigo, 'cpf_cliente'] = novo
# Junção das tuas tabelas estabelecidas no CRUD e refino na diferença de colunas (tabela.conta = cpf_cliente) e (tabela.cliente = cpf)
df_completo = pd.merge(df_conta, df_cliente, left_on="cpf_cliente", right_on="cpf")

df_completo = df_completo.drop(columns=["cpf_cliente"])

display(df_completo.head(5))

,id_x,numero,saldo,id_y,nome,cpf
0,1,34828,10980.86,1,Bruno Raphael Andrade Santos,615.019.033-14
1,2,63594,2236.49,2,Maria Victoria,123.456.789-10
2,3,5276,5373.46,3,Pedro Miguel,013.179.063.30
3,4,21717,4301.14,4,Kaique Pires,620.983.145-15
4,5,13005,1421.19,5,Brayan Duarte,269.451.730-70


In [11]:
# Conferindo quais os tipos de dados temos aqui e extraindo algumas informações relevantes
def extracao_info (df) : 
    print ("-=" * 30)
    print ("Aqui podemos conferir o tot rows and lines")
    print(df.shape)
    print ("-=" * 30)
    print (" INFORMAÇÕES SOBRE A TIPAGEM DOS DADOS")
    df.info() 
    print ("-=" *30)
    print ("PRINCIPAIS MÉTRICAS ESTATÍSTICAS")
    print(df.describe())

extracao_info(df_completo)



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Aqui podemos conferir o tot rows and lines
(1012, 6)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
 INFORMAÇÕES SOBRE A TIPAGEM DOS DADOS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id_x    1012 non-null   int64  
 1   numero  1012 non-null   int64  
 2   saldo   1012 non-null   float64
 3   id_y    1012 non-null   int64  
 4   nome    1012 non-null   object 
 5   cpf     1012 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 47.6+ KB
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
PRINCIPAIS MÉTRICAS ESTATÍSTICAS
              id_x        numero         saldo         id_y
count  1012.000000   1012.000000   1012.000000  1012.000000
mean    506.500000  51182.454545   2514.466146   508.047431
std     292.283538  28502.041601   1512.861532 